In [35]:
# AI Model Implementation in Jupyter Notebook

## Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
import sqlite3
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [36]:
## Cell 2: Correctly connect to SQLite database and load data
import sqlite3
import pandas as pd

# Absolute path to your SQLite database (copy this exactly)
db_path = r"C:\Users\HP\Documents\GitHub\vroomble\Vroomble Dataset\prediction_database.db"

# Connect explicitly
conn = sqlite3.connect(db_path)

# Load data from SQLite tables into DataFrames
car_model_data = pd.read_sql_query("SELECT * FROM Car_Model", conn)
car_modifications_data = pd.read_sql_query("SELECT * FROM Car_Modifications", conn)

# Verify loaded data
print("Car Model Data:")
print(car_model_data.head())

print("\nCar Modifications Data:")
print(car_modifications_data.head())


Car Model Data:
         Make     Model    Month  Base_Price_PHP  Monthly_Inflation_Rate
0       Honda      CR-V  2024-03         2200000                    0.05
1       Honda     Civic  2024-03         1600000                    0.05
2        Ford   Everest  2024-03         2100000                    0.05
3      Toyota  Fortuner  2024-03         1900000                    0.05
4  Mitsubishi    Mirage  2024-03          800000                    0.05

Car Modifications Data:
    Make Model    Month Modification_Type  \
0  Honda  CR-V  2023-01     Street/Casual   
1  Honda  CR-V  2023-01           Offroad   
2  Honda  CR-V  2023-01           Offroad   
3  Honda  CR-V  2023-01           Offroad   
4  Honda  CR-V  2023-01           Offroad   

                               Car_Part  Modification_Cost_PHP  \
0   Honda CR-V Chrome Door Handle Cover                   5000   
1      Honda CR-V Heavy-Duty Skid Plate                  15000   
2     Honda CR-V Off-Road Fender Flares             

In [37]:
# Cell 3: Prepare training data for the AI model

# Initialize feature and target lists
X = []  # Features: Base price, Modification Cost, Inflation rate
y = []  # Target: Final Price

# Iterate through each row to prepare the dataset
for _, row in car_model_data.iterrows():
    base_price = row["Base_Price_PHP"]
    inflation_rate = row["Monthly_Inflation_Rate"]
    
    # Compute total modification cost for the model
    modifications = car_modifications_data[
        (car_modifications_data["Model"] == row["Model"])
    ]["Modification_Cost_PHP"].sum()
    
    # Append features and target values
    X.append([base_price, modifications, inflation_rate])
    y.append(base_price + modifications)

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Verify prepared data
print("Features (X):", X[:5])
print("Target (y):", y[:5])


Features (X): [[2.20e+06 4.98e+06 5.00e-02]
 [1.60e+06 4.98e+06 5.00e-02]
 [2.10e+06 4.98e+06 5.00e-02]
 [1.90e+06 4.98e+06 5.00e-02]
 [8.00e+05 4.98e+06 5.00e-02]]
Target (y): [7180000 6580000 7080000 6880000 5780000]


In [38]:
# Cell 4: Scale features, train Linear Regression model, and save scaler/model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import joblib

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train Linear Regression model
model = LinearRegression()
model.fit(X_scaled, y)

# Output trained parameters
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

# Save the scaler and trained model explicitly
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(model, 'trained_model.pkl')

print("✅ Scaler and trained model saved successfully.")


Intercept: 2966009.6326530613
Coefficients: [ 496837.14335995 1852526.7522453        0.        ]
✅ Scaler and trained model saved successfully.


In [39]:
## Cell 5: Prediction function using the trained model
def predict_price(make, model_name, modification_type, selected_parts, months=12):
    # Fetch the base price and inflation rate
    query_model = """
        SELECT Base_Price_PHP, Monthly_Inflation_Rate FROM Car_Model
        WHERE Make = ? AND Model = ?
    """
    car_info = pd.read_sql_query(query_model, conn, params=(make, model_name))

    if car_info.empty:
        print(f"❌ Error: '{make} {model_name}' not found in database.")
        return None

    base_price = car_info.iloc[0]["Base_Price_PHP"]
    inflation_rate = car_info.iloc[0]["Monthly_Inflation_Rate"]

    # Check if selected_parts is empty
    if not selected_parts:
        modification_cost = 0
    else:
        placeholders = ','.join('?' for _ in selected_parts)
        query_mod = f"""
            SELECT SUM(Modification_Cost_PHP) as total_mod_cost FROM Car_Modifications
            WHERE Modification_Type = ?
            AND Model = ?
            AND Car_Part IN ({placeholders})
        """
        params = [modification_type, model_name] + selected_parts
        modification_cost_df = pd.read_sql_query(query_mod, conn, params=params)
        
        modification_cost = modification_cost_df.iloc[0]['total_mod_cost']

        # Handle if no modifications found
        if modification_cost is None:
            print("⚠️ Warning: No matching modifications found. Defaulting modification cost to 0.")
            modification_cost = 0

    # Inflation calculation
    future_inflation = (1 + inflation_rate) ** months

    # Load scaler and trained model
    scaler = joblib.load('scaler.pkl')
    model = joblib.load('trained_model.pkl')

    # Scale inputs
    features_scaled = scaler.transform([[base_price * future_inflation, modification_cost, inflation_rate]])

    # Predict price
    predicted_price = model.predict(features_scaled)[0]

    # Display prediction result
    result = {
        "Make": make,
        "Model": model_name,
        "Modification Type": modification_type,
        "Selected Car Parts": selected_parts,
        "Base Price (PHP)": base_price,
        "Car Parts Cost (PHP)": modification_cost,
        "Current Total Price (PHP)": base_price + modification_cost,
        f"Predicted Price After {months} Months (PHP)": predicted_price
    }

    for key, value in result.items():
        print(f"{key}: {value}")

    return result


In [40]:
## Cell 6: Function to predict car price based on selected parts
# Example usage
test_result = predict_price(
    make='Toyota',
    model_name='Vios',
    modification_type='Street/Casual',
    selected_parts=['Toyota Vios Trunk Lip Spoiler','Toyota Vios Chrome Door Handle Cover'],
    months=12
)



Make: Toyota
Model: Vios
Modification Type: Street/Casual
Selected Car Parts: ['Toyota Vios Trunk Lip Spoiler', 'Toyota Vios Chrome Door Handle Cover']
Base Price (PHP): 750000.0
Car Parts Cost (PHP): 276000
Current Total Price (PHP): 1026000.0
Predicted Price After 12 Months (PHP): 1622892.2445165974


In [41]:
conn.close()